In [3]:
import java.nio.file.Files
import java.nio.file.Paths


val realInput = Files.readString(Paths.get("input.txt"))
val testInput = """aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out
"""
val testInput2 = """svr: aaa bbb
aaa: fft
fft: ccc
bbb: tty
tty: ccc
ccc: ddd eee
ddd: hub
hub: fff
eee: dac
dac: fff
fff: ggg hhh
ggg: out
hhh: out"""

val data = realInput

val input = data.trim().split('\n').map {
    val a = it.split(":")
    a[0] to a[1].trim().split(" ")
}.toMap()
input.entries.first()

the=[taq, aki]

# First approach: trivial BFS for finding path from you -> out

In [4]:
data class State(val node: String, val visitedNodes: List<String>)

fun findPaths(input: Map<String, List<String>>, from: String, to: String) : Int {
    val queue = ArrayDeque<State>()

    val initialState = State(from, listOf(from))
    val targetNode = to
    val memo = mutableMapOf<String, Int>()
    queue.add(initialState)

    var paths = 0
    while (queue.isNotEmpty()) {
        //print("$memo,")
        val current = queue.removeFirst()

        if (memo.getOrDefault(current.node, 0) > 0 || current.node == targetNode) {
            //println("in memo or target")
            for (n in current.visitedNodes) { memo[n] = memo.getOrDefault(n, 0) + 1 }
            paths++
        } else {
            //println(input[current.node])
            for (node in input[current.node] ?: emptyList()) {
                if (!current.visitedNodes.contains(node)) {
                    //println("${current.node} -> $node")
                    queue.add(State(node, current.visitedNodes + node))
                }
            }
        }
    }
    //println(memo)
    return paths
}

findPaths(input, "you", "out")


708

# Part2 cannot be solved using BFS (heap space)
Switch to a simple DFS (depth first search) with memoization. Since there are no loops in the graph we can ommit some safety.
Also works for the first solution


In [5]:
fun findPathsRecursive(input: Map<String, List<String>>, from: String, to: String): Long {
    // stores how many paths exist from each node to the target
    val memo = mutableMapOf<String, Long>()

    fun dfs(node: String): Long {
        // reached the target, this counts as 1 valid path
        if (node == to) return 1
        // already computed paths from this node, return cached result
        memo[node]?.let { return it }

        val paths = input[node]
            ?.sumOf { dfs(it) } // Sum paths from all children
            ?: 0                // No children = no paths (dead end)

        // put into cache
        memo[node] = paths
        return paths
    }

    return dfs(from)
}

findPathsRecursive(input, "you", "out")

708

# There are only to possible solutions possible

Quote from the description
```
[...] so they now need you to find every path from svr (the server rack) to out.
However, the paths you find must all also visit both dac and fft (in any order).
```

So it's either svr->dac->fft->out or svr->fft->dac->out.



In [6]:
val svr_dac = findPathsRecursive(input, "svr", "dac")
val svr_fft = findPathsRecursive(input, "svr", "fft")
val dac_fft = findPathsRecursive(input, "dac", "fft")
val fft_dac = findPathsRecursive(input, "fft", "dac")
val dac_out = findPathsRecursive(input, "dac", "out")
val fft_out = findPathsRecursive(input, "fft", "out")
val svr_dac_fft_out = svr_dac * dac_fft * fft_out
val svr_fft_dac_out = svr_fft * fft_dac * dac_out
println("svr->dac->fft->out: $svr_dac_fft_out = $svr_dac * $dac_fft * $fft_out")
println("svr->fft->dac->out: $svr_fft_dac_out = $svr_fft * $fft_dac * $dac_out")

max(svr_dac_fft_out, svr_fft_dac_out)

svr->dac->fft->out: 0 = 2438626827156 * 0 * 2305764038056
svr->fft->dac->out: 545394698933400 = 5477 * 20531772 * 4850


545394698933400